## Librerias

In [26]:
import matplotlib
matplotlib.use('Agg') # Fuerza un backend que no requiere ventana gráfica

# ... el resto de tus imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# --- IMPORTAR LOS GLADIADORES ---
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


## carga de archivos

In [27]:
df = pd.read_csv('DATASET_MAESTRO_ENTRENAMIENTO.csv') # Tu archivo final
df.head(5)

,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,...,P14,P15,P16,P17,P18,P19,P20,Carrera_Target,Satisfaccion,Tipo_Dato
0,a,d,a,a,d,a,d,a,a,a,...,a,a,a,a,a,a,d,Ingeniería de Software,3,Real
1,d,a,a,a,a,d,d,c,a,b,...,b,d,a,a,a,a,a,Ingeniería de Software,5,Real
2,d,a,a,a,a,a,d,d,d,e,...,d,a,b,a,a,d,d,Ingeniería de Software,5,Real
3,a,a,a,a,c,a,a,d,a,a,...,a,a,a,a,d,a,a,Ingeniería de Software,5,Real
4,a,a,c,a,d,a,a,a,e,a,...,a,c,c,a,a,a,a,Ingeniería de Software,4,Real


## Preprocesamiento (Codificar letras a números)
### IMPORTANTE: SVM y KNN son sensibles a la escala, así que a veces ayuda escalar, pero como todas tus preguntas son del 0 al 4 (incisos), ya están en la misma escala.

In [28]:
for col in df.columns:
    if col.startswith('P') or col == 'Carrera_Target':
        le = LabelEncoder()
        df[col] = le.fit_transform(df[col])

X = df.drop(['Carrera_Target', 'Tipo_Dato', 'Satisfaccion'], axis=1, errors='ignore')
y = df['Carrera_Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [29]:
X.head()

,P01,P02,P03,P04,P05,P06,P07,P08,P09,P10,P11,P12,P13,P14,P15,P16,P17,P18,P19,P20
0,0,3,0,0,3,0,3,0,0,0,0,1,0,0,0,0,0,0,0,3
1,3,0,0,0,0,3,3,2,0,1,1,3,0,1,3,0,0,0,0,0
2,3,0,0,0,0,0,3,3,3,4,0,1,0,3,0,1,0,0,3,3
3,0,0,0,0,2,0,0,3,0,0,0,0,0,0,0,0,0,3,0,0
4,0,0,2,0,3,0,0,0,4,0,1,3,0,0,2,2,0,0,0,0


## Definir la lista de modelos

In [30]:
modelos = [
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('SVM (Kernel RBF)', SVC(gamma='auto')), # Bueno para fronteras curvas
    ('KNN (5 Vecinos)', KNeighborsClassifier(n_neighbors=5)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
    ('Naive Bayes', GaussianNB()) # El más rápido y probabilístico
]

resultados = []
nombres = []

print(f"{'Modelo':<20} | {'Precisión':<10}")
print("-" * 35)

Modelo               | Precisión 
-----------------------------------


## Entrenar y Evaluar uno por uno

In [31]:
for nombre, modelo in modelos:
    # Entrenar
    modelo.fit(X_train, y_train)
    
    # Predecir
    y_pred = modelo.predict(X_test)
    
    # Calcular precisión
    acc = accuracy_score(y_test, y_pred)
    
    resultados.append(acc)
    nombres.append(nombre)
    
    print(f"{nombre:<20} | {acc:.2%}")

Random Forest        | 55.45%
SVM (Kernel RBF)     | 51.14%
KNN (5 Vecinos)      | 34.32%
Gradient Boosting    | 57.61%
Naive Bayes          | 43.41%


## Graficar los resultados

In [32]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de la gráfica
plt.figure(figsize=(10, 6))
colores = sns.color_palette("viridis", len(modelos))

# Usamos hue=nombres y legend=False para cumplir con la nueva versión de Seaborn
sns.barplot(x=nombres, y=resultados, hue=nombres, palette=colores, legend=False)

plt.ylim(0.7, 1.0) 
plt.title('Comparación de Modelos: Precisión por Algoritmo', fontsize=14)
plt.ylabel('Precisión (Accuracy)')
plt.xticks(rotation=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Agregar las etiquetas de porcentaje sobre las barras
for i, v in enumerate(resultados):
    plt.text(i, v + 0.01, f"{v:.1%}", ha='center', fontweight='bold')

plt.tight_layout()

# LA LÍNEA MÁS IMPORTANTE:
plt.savefig('resultados_torneo_modelos.png', dpi=300) # dpi=300 para que se vea súper nítida
print("¡Éxito! La gráfica se guardó como 'resultados_torneo_modelos.png' en tu carpeta actual.")

# Si estás en VS Code, esto ayudará a que se vea en el notebook también
plt.show()

¡Éxito! La gráfica se guardó como 'resultados_torneo_modelos.png' en tu carpeta actual.


/tmp/ipykernel_27100/471040450.py:28: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()
